# MASTER Classification Project

By Ednalyn C. De Dios & Michael P. Moran

## Project Planning

### Goals
1. Explain what is driving customers to churn.

### Questions Presented & Answers
1. Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
1. Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
1. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
1. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

### Data Dictionary & Domain Knowledge

### gender

| Description | gender
| ------------- |-------------
| Female      | 0
| Male      | 1

### phone_id

| Phone Service | phone_id
| ------------- |-------------
| No phone      | 0
| One Line      | 1
| Two+ lines    | 2

### household_type_id

| Partner & Dependents      | household_type_id
| -------------             |-------------
| No partner, no dependents      | 0
| Partner, no dependents      | 1
| No partner, dependents    | 2
| Partner, dependents | 3

### streaming_services

| Streaming TV & Streaming Movies      | streaming_services
| -------------             |-------------
| No internet service      | 0
| No streaming tv, no streaming movies      | 1
| Has streaming tv, no streaming movies    | 2
| No streaming tv, has streaming movies | 3
| Has streaming tv, has streaming movies | 4

### online_security_backup

| Online Security & Online Backup      | online_security_backup
| -------------             |-------------
| No internet service      | 0
| No security, no backup | 1
| Has security, no backup | 2
| No security, has backup | 3
| Has security, has backup | 4

### device_protection
| Device Protection | Value
| ------ |-----
| No internet service | 1
| No | 0
| Yes | 2

### tech_support
| Tech Support | Value
| ------ |-----
| No internet service | 1
| No | 0
| Yes | 2

### paperless_billing
| Paperless Billing | Value
| ------ |-----
| No | 0
| Yes | 1

### internet_service_type
| internet_service_type_id | internet_service_type |
| ------ |-----|
|                        0 | None                  |
|                        1 | DSL                   |
|                        2 | Fiber optic           |


### payment types
| payment_type_id | payment_type              |
| ------ |-----
|               1 | Electronic check          |
|               2 | Mailed check              |
|               3 | Bank transfer (automatic) |
|               4 | Credit card (automatic)   |


### contract_type_id
| contract_type_id | contract_type  |
| ------ |-----
|                1 | Month-to-month |
|                2 | One year       |
|                3 | Two year       |

### Hypotheses

## Prepare Environment

In [ ]:
from env import host, user, password
import adalib

import numpy as np
import pandas as pd

from sqlalchemy import create_engine

import scipy.stats as stats
    
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.tools.plotting import scatter_matrix
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter

## Acquisition

### Grab Data

In [ ]:
def get_db_url(
    hostname: str, username: str, password: str, db_name: str
) -> str:
    """
    return url for accessing a mysql database
    """
    return f"mysql+pymysql://{username}:{password}@{hostname}/{db_name}"


def get_sql_conn(hostname: str, username: str, password: str, db_name: str):
    """
    return a mysql connection object
    """
    return create_engine(get_db_url(host, user, password, db_name))


def df_from_sql(query: str, url: str) -> pd.DataFrame:
    """
    return a Pandas DataFrame resulting from a sql query
    """
    return pd.read_sql(query, url)


def get_telco_data() -> pd.DataFrame:
    db = "telco_churn"
    query = ("SELECT * "
             f"FROM customers;")
    url = get_db_url(host, user, password, db)
    return df_from_sql(query, url)

In [ ]:
df = get_telco_data()

### Summarize Data

In [ ]:
def peekatdata(dataframe):
    head_df = dataframe.head()
    print(f"HEAD\n{head_df}", end="\n\n")

    tail_df = dataframe.tail()
    print(f"TAIL\n{tail_df}", end="\n\n")

    shape_tuple = dataframe.shape
    print(f"SHAPE: {shape_tuple}", end="\n\n")

    describe_df = dataframe.describe()
    print(f"DESCRIPTION\n{describe_df}", end="\n\n")

    print(f"INFORMATION")
    dataframe.info()

In [ ]:
peekatdata(df)

## Data Prep

1. We're computing and returning the values by frequency for each variable.

In [ ]:
def df_value_counts(dataframe):
    for col in df.columns:
        n = df[col].unique().shape[0]
        col_bins = min(n, 10)
        print(f"{col}:")
        if df[col].dtype in ['int64', 'float64'] and n > 10:
            print(df[col].value_counts(bins=col_bins, sort=False))
        else:
            print(df[col].value_counts())
        print("\n")

In [ ]:
df_value_counts(df)

### Observations from Value Counts
- customer_id has no duplicates
- gender is about even
- customers are mostly not seniors
- about equally split along single/partner
- most customers do not have dependents
- there are many new and many old customers
- overwhelming majority have phone service
- closely split along multiple_lines
- overwhelming majority have internet service
    - more have fiber than DSL
    - most do not have online_security
    - most do not have online_backup
    - most do not have device_protection
    - most do not have tech_support
- billing
    - about evenly split along streaming_tv
    - about evenly splot along streaming_movies
    - most customers are month-to-month
    - most customers are paperless billing
    - most customers pay by some form of check
    - many customers pay less than $30
    - most are pay between \\$45 and \\$110 per month
- most have not churned
    - about 1900 have
    

### Handle Missing Values

In [ ]:
def df_missing_vals(dataframe):
    null_count = dataframe.isnull().sum()
    null_percentage = (null_count / dataframe.shape[0]) * 100
    empty_count = pd.Series(((dataframe == " ") | (dataframe == "")).sum())
    return pd.DataFrame({"nmissing": null_count, "percentage": null_percentage, "nempty": empty_count})

In [ ]:
print(df_missing_vals(df))

#### Takeaway
- No columns have NaNs.
- 11 observations have " " value for total_charges
    - Replace with monthly_charges * tenure
    - Those 11 values all had a tenure of 0 so they were essentially replaced with zeroes.

In [ ]:
df['total_charges'] = np.where(df['total_charges'] == ' ', (df.monthly_charges * df.tenure), df['total_charges'])

3. We're transforming churn such that "yes" = 1 and "no" = 0

In [ ]:
df['churn'] = df['churn'].map( {'No': 0, 'Yes': 1} ).astype(int)

4. We created a new column, tenure_year, that is a result of translating tenure from months to years.

In [ ]:
df['tenure_year'] = df.tenure / 12

5. We created a new column, phone_id, that combines phone_service and multiple_lines.

In [ ]:
def add_phone_id(df):
    '''
    return DataFrame with a new colum phone_id that combines phone_service and multiple_lines as follows:
    
    # 0 = no phone service
    # 1 = One line
    # 2 = Two+ lines
    
    '''
    df_temp = df.copy()
    df_temp.loc[(df_temp.phone_service == 'No'), 'phone_id'] = 0
    df_temp.loc[(df_temp.phone_service == 'Yes') & (df_temp.multiple_lines == 'No'), 'phone_id'] = 1
    df_temp.loc[(df_temp.phone_service == 'Yes') & (df_temp.multiple_lines == 'Yes'), 'phone_id'] = 2
    df_temp = df_temp.astype({'phone_id': int})
    return df_temp

df_sql = add_phone_id(df)

6. We add new column household_type_id that combines partner and dependents.

In [ ]:
def add_household_type_id(df):
    '''
    return DataFrame with a new colum household_type_id that combines partner and dependents as follows:
    
    # 0 = no partner, no dependents
    # 1 = has partner, no dependents
    # 2 = no partner, has dependents
    # 3 = has partner, has dependents
    
    '''
    df_temp = df.copy()
    df_temp.loc[(df_temp.partner == 'No')& (df_temp.dependents == 'No'), 'household_type_id'] = 0
    df_temp.loc[(df_temp.partner == 'Yes') & (df_temp.dependents == 'No'), 'household_type_id'] = 1
    df_temp.loc[(df_temp.partner == 'No') & (df_temp.dependents == 'Yes'), 'household_type_id'] = 2
    df_temp.loc[(df_temp.partner == 'Yes') & (df_temp.dependents == 'Yes'), 'household_type_id'] = 3
    df_temp = df_temp.astype({'household_type_id': int})
    return df_temp

df_sql = add_household_type_id(df_sql)

7. We created a new column streaming_services that combines both streaming services for tv and movies.

In [ ]:
def add_streaming_services(df):
    """
    return DataFrame with a new column streaming_services that combines streaming_tv and streaming_movies columns as follows:
    # 0 = no internet service
    # 1 = no streaming_tv, no streaming_movies
    # 2 = has streaming_tv, no streaming_movies
    # 3 = no streaming_tv, has streaming_movies
    # 4 = has streaming_tv, has streaming_movies
    """
    df_temp = df.copy()
    df_temp.loc[(df_temp.streaming_tv == "No internet service") & (df_temp.streaming_movies == 'No internet service'), "streaming_services"] = int(0)
    df_temp.loc[(df_temp.streaming_tv == "No") & (df_temp.streaming_movies == 'No'), "streaming_services"] = int(1)
    df_temp.loc[(df_temp.streaming_tv == "Yes") & (df_temp.streaming_movies == 'No'), "streaming_services"] = int(2)
    df_temp.loc[(df_temp.streaming_tv == "No") & (df_temp.streaming_movies == 'Yes'), "streaming_services"] = int(3)
    df_temp.loc[(df_temp.streaming_tv == "Yes") & (df_temp.streaming_movies == 'Yes'), "streaming_services"] = int(4)
    df_temp = df_temp.astype({"streaming_services": int})
    
    return df_temp

df_sql = add_streaming_services(df_sql)

8. We created a new column online_security_backup that combines both online security and backup.

In [ ]:
def add_online_security_backup(df):
    """
    return DataFrame with a new column streaming_services that combines online_security and online_backup columns as follows:
    # 0 = no internet service
    # 1 = no online_security, no online_backup
    # 2 = has online_security, no online_backup
    # 3 = no online_security, has online_backup
    # 4 = has online_security, has online_backup
    """
    df_temp = df.copy()
    df_temp.loc[(df_temp.online_security == "No internet service") & (df_temp.online_backup == "No internet service"), "online_security_backup"] = 0
    df_temp.loc[(df_temp.online_security == "No") & (df_temp.online_backup == "No"), "online_security_backup"] = 1
    df_temp.loc[(df_temp.online_security == "Yes") & (df_temp.online_backup == "No"), "online_security_backup"] = 2
    df_temp.loc[(df_temp.online_security == "No") & (df_temp.online_backup == "Yes"), "online_security_backup"] = 3
    df_temp.loc[(df_temp.online_security == "Yes") & (df_temp.online_backup == "Yes"), "online_security_backup"] = 4
    df_temp = df_temp.astype({"online_security_backup": int})
    
    return df_temp

df_sql = add_online_security_backup(df_sql)

Change value for "None" internet_service_type_id from 3 to 0. It is more intuitive and makes correlation tests easier to run.

In [ ]:
def fix_internet_service_type_id(df):
    """
    0 = no internet
    1 = DSL
    2 = fiber
    """
    df_temp = df.replace({"internet_service_type_id": 3}, 0)
    return df_temp

df_sql = fix_internet_service_type_id(df_sql)

9. Data Split
    - We Split data into train (70%) & test (30%) samples.

In [ ]:
xcols = ['customer_id', 'gender', 'senior_citizen', 'internet_service_type_id', 'device_protection', 'tech_support',
         'contract_type_id', 'paperless_billing', 'payment_type_id', 'monthly_charges', 'total_charges',
         'tenure_year', 'phone_id', 'household_type_id', 'streaming_services', 'online_security_backup']
X = df_sql[xcols]
y = df_sql[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123, stratify=y)

10. Variable Encoding

    - We encode for each non-numeric variable.

In [ ]:
to_encode = ["gender", "device_protection", "tech_support", "paperless_billing"]
encoders = {}
for col in to_encode:
    encoder = LabelEncoder()
    encoder.fit(X_train[col])
    X_train[col] = encoder.transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])
    encoders[col] = encoder

11. Numeric Scaling

    - We standardize numerical variables.
    - [ ] include tenure_year if time allows.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train[["monthly_charges", "total_charges"]])
X_train[["monthly_charges", "total_charges"]] = scaler.transform(X_train[["monthly_charges", "total_charges"]])
X_test[["monthly_charges", "total_charges"]] = scaler.transform(X_test[["monthly_charges", "total_charges"]])

Make train_df and test_df

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

## Data Exploration

### The target variable: 'churn'

In [ ]:
train_df.churn.value_counts().sort_index().plot(kind = "bar",alpha = .5)
plt.title("Distribution of Customers Who Churned")
plt.grid(b = True , which = "major")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
df_de1 = y_train
df_de1 = pd.concat([df_de1, pd.cut(X_train["tenure_year"], 6)], axis=1)
sns.barplot(x=df_de1.tenure_year, y=df_de1.churn, alpha=0.5)
plt.title("Churn Rate by Tenure Cohort")
plt.show()

In [ ]:
sns.distplot(train_df.tenure_year)

**1. Could the month in which customers signed up influence churn?**

- Churn is highest for customers with a tenure_year of less than or equal to 1, followed by those with a tenure_year greater than 1 but less than or equal to 2. There is a group of customers nearing tenure_year 6 with a fairly high churn rate.

**2. Are there features that indicate a higher propensity to churn?**

_Services/traits/demographic that indicate a higher propensity for churning:_

- Fiber optic customers have the highest churn rate, followed by DSL customers. Those with no internet have the lowest churn rate of the three groups
- Churn rate is pretty equal among phone_id. I think this mean that the type of phone service is not driving or connected with churn.
- Those without security or backup have the highest churn rate, followed by those with backup only.
- Seniors have a higher churn rate than non-seniors
- Those without device protection have a churn rate above the mean compared to those without
- Those with a "no" for tech_support have a churn rate above the mean compared to those with a "yes"
- Gender is not driving churn. No surprise here.
- Month-to-month contracts have a churn rate above the mean, and they churn much more than other contract types.
- Paperless billing have a churn rate above the mean
- E-check have a churn rate above the mean
- Those with no partner and no dependents have a churn rate above the mean; those with a partner and dependents have the lowest churn rate
- Those with streaming services have an above average churn rate

In [ ]:
barplot_features = ['internet_service_type_id', 'phone_id', 'online_security_backup', 'senior_citizen', 'device_protection', 'tech_support',
           'gender', 'contract_type_id', 'paperless_billing', 'payment_type_id', 'household_type_id', 'streaming_services']
churn_rate = y_train.churn.mean()
plt.figure(figsize=(15, 30))
for i, feature in enumerate(barplot_features):
    plt.subplot(6, 2, i + 1)
    ax = sns.barplot(X_train[feature], y_train.churn, alpha=.5)
    ax.set_yticklabels(['{:,.0%}'.format(y) for y in ax.get_yticks()])
    plt.ylabel('Churn Rate')
    plt.axhline(churn_rate, ls='--', color='grey')
plt.show()

**3. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point?**

- Internet
    - For no internet customers, there does not appear to be a price threshold that triggers churn
    - For DSL customers, it appears that a standardized threshold between 0.2 and 0.4 triggers some churn
    - For fiber customers, there does not appear to be a price threshold that triggers churn
- Phone
    - For no phone customers, customers paying a lower price have a more churn
    - For one line customers, churn increases after 0.5 standardized monthly price
    - For two+ line customers, churn increases after 0.6 standardized monthly price
- Online security & backup
    - There does not appear to be a price threshold that triggers churn for these customers
- Streaming services
    - Churn increases after 0.5 or 0.6 standardized price threshold for those with some kind of streaming service

In [ ]:
vplot_features = ['internet_service_type_id', 'phone_id', 'online_security_backup', 'streaming_services']

plt.figure(figsize=(15, 15))
for i, feature in enumerate(vplot_features):
    plt.subplot(2, 2, i + 1)
    sns.violinplot(feature, 'monthly_charges', hue='churn',
        data=train_df, split=True, palette=['blue','orange'])

In [ ]:
features = ['paperless_billing', 'payment_type_id', 'household_type_id']
feature_count = range(len(features))

_, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for i, feature in enumerate(features):
    sns.violinplot(feature, 'monthly_charges', hue='churn',
        data=train_df, split=True, ax=ax[i], palette=['blue','orange'])

**4. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?**

- Those customers on a month-to-month have a higher churn rate even when excluding newer customers

In [ ]:
df_de4 = train_df[(train_df.tenure_year > 1) & ((train_df.contract_type_id == 1) | (train_df.contract_type_id == 2))][["contract_type_id", "churn"]]
sns.barplot(x=df_de4.contract_type_id, y=df_de4.churn, alpha=0.5)
plt.title("Churn Rate for Month-to-Month and One-year Contract After First Year")
plt.show()

**5. is the mean monthly_charges of those who have churned significantly different from that of those who have not churned?**

In [ ]:
ax = sns.boxplot(x='churn', y='monthly_charges', data=train_df)

In [ ]:
features = ['senior_citizen', 'internet_service_type_id', 'device_protection']

_, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,5))

churn_rate = train_df.churn.mean()

for i, feature in enumerate(features):
    sns.barplot(feature, 'churn', data=train_df, ax=ax[i], alpha=.5)
    ax[i].set_ylabel('Churn Rate')
    ax[i].axhline(churn_rate, ls='--', color='grey')

In [ ]:
features = ['tech_support', 'contract_type_id', 'paperless_billing']

_, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,5))

churn_rate = train_df.churn.mean()

for i, feature in enumerate(features):
    sns.barplot(feature, 'churn', data=train_df, ax=ax[i], alpha=.5)
    ax[i].set_ylabel('Churn Rate')
    ax[i].axhline(churn_rate, ls='--', color='grey')

In [ ]:
features = ['phone_id', 'household_type_id', 'streaming_services']

_, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,5))

churn_rate = train_df.churn.mean()

for i, feature in enumerate(features):
    sns.barplot(feature, 'churn', data=train_df, ax=ax[i], alpha=.5)
    ax[i].set_ylabel('Churn Rate')
    ax[i].axhline(churn_rate, ls='--', color='grey')

In [ ]:
features = ['online_security_backup']

_, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,5))

churn_rate = train_df.churn.mean()

for i, feature in enumerate(features):
    sns.barplot(feature, 'churn', data=train_df, ax=ax[i], alpha=.5)
    ax[i].set_ylabel('Churn Rate')
    ax[i].axhline(churn_rate, ls='--', color='grey')

**6. How much of monthly_charges can be explained by internet_service_type?**

In [ ]:
pd.crosstab(train_df.internet_service_type_id, train_df.churn, margins=True).style.background_gradient(cmap='YlOrRd')

**7. How much of monthly_charges can be explained by internet_service_type + phone service type (0, 1, or multiple lines).**

**8. Create visualizations exploring the interactions of variables** (independent with independent and independent with dependent). The goal is to **identify features that are related to churn**

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(train_df.corr(), cmap='Blues', annot=True)

## Modeling

1. Feature Selection: Are there any variables that seem to provide limited to no additional information? If so, remove those and assign the new limited dataframe to train_reduced

2. Train (fit, transform, evaluate) a logistic regression model varying your meta-parameters.

3. Compare evaluation metrics across all the models, and select the best performing model.

4. Test the final model (transform, evaluate) on your out-of-sample data (test_df). Summarize the performance. Interpret your results.

# Delivery

1. I will need you to deliver to me a csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn). I would also like a single google slide that illustrates how your model works, including the features being used, so that I can deliver this to the SLT when they come with questions about how these values were derived. Please make sure you include how likely your model is to give a high probability of churn when churn doesn't occur, to give a low probability of churn when churn occurs, and to accurately predict churn.

1. Finally, our development team will need a .py file that will take in a new dataset, (in the exact same form of the one you acquired from telco_churn.customers) and perform all the transformations necessary to run the model you have developed on this new dataset to provide probabilities and predictions.